<a href="https://colab.research.google.com/github/Codequirk/CodSoft/blob/main/task_1_Titanic_survival_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TITANIC SURVIVAL PREDICTION PROJECT


In [5]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive')

df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Titanic-Dataset.csv")
df.head()

Mounted at /content/drive


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [7]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [8]:
df.isnull().sum()

,0
PassengerId,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,177
SibSp,0
Parch,0
Ticket,0
Fare,0


In [9]:
df["Age"]=df["Age"].fillna(df["Age"].median())
df["Embarked"]=df["Embarked"].fillna(df["Embarked"].mode()[0])
if "Cabin" in df.columns:
  df=df.drop(columns=["Cabin"])


for col in ['Passengers','Name','Ticket']:
  if col in df.columns:
    df=df.drop(columns=[col])
df=pd.get_dummies(df,columns=["Sex","Embarked"],drop_first=True)
df.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S
0,1,0,3,22.0,1,0,7.2500,True,False,True
1,2,1,1,38.0,1,0,71.2833,False,False,False
2,3,1,3,26.0,0,0,7.9250,False,False,True
3,4,1,1,35.0,1,0,53.1000,False,False,True
4,5,0,3,35.0,0,0,8.0500,True,False,True


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report

X=df.drop("Survived",axis=1)
y=df["Survived"]

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
model=LogisticRegression(max_iter=1000)
model.fit(X_train,y_train)
y_pred=model.predict(X_test)
accuracy=accuracy_score(y_test,y_pred)
print(f"accurscy on test set is {accuracy:.2f}")
print("confusion matrix:")
print(confusion_matrix(y_test,y_pred))
print('\nClassification report:')
print(classification_report(y_test,y_pred))



accurscy on test set is 0.80
confusion matrix:
[[89 16]
 [19 55]]

Classification report:
              precision    recall  f1-score   support

           0       0.82      0.85      0.84       105
           1       0.77      0.74      0.76        74

    accuracy                           0.80       179
   macro avg       0.80      0.80      0.80       179
weighted avg       0.80      0.80      0.80       179



FEATURE ENGINEERING


In [11]:
df['FamilySize']=df['SibSp']+df['Parch']+1

In [12]:
df['isAlone']=1
df.loc[df['FamilySize']>1,'isAlone']=0

In [13]:
df['ClassFare']=df['Pclass']*df['Fare']

In [14]:
df['AgeClass']=df['Age']*df['Pclass']

In [15]:
df['FareBand']=pd.qcut(df['Fare'],4,labels=[1,2,3,4])

In [16]:
bins=[0,10,20,40,60,80]
labels=['child','teen','adult','middleage','senior']
df['AgeGroup']=pd.cut(df['Age'],bins=bins,labels=labels)
df=pd.get_dummies(df,columns=['AgeGroup'],drop_first=True)

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report

X=df.drop("Survived",axis=1)
y=df["Survived"]

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
model=LogisticRegression(max_iter=5000)
model.fit(X_train,y_train)
y_pred=model.predict(X_test)
accuracy=accuracy_score(y_test,y_pred)
print(f"accurscy on test set is {accuracy:.2f}")
print("confusion matrix:")
print(confusion_matrix(y_test,y_pred))
print('\nClassification report:')
print(classification_report(y_test,y_pred))

accurscy on test set is 0.80
confusion matrix:
[[89 16]
 [20 54]]

Classification report:
              precision    recall  f1-score   support

           0       0.82      0.85      0.83       105
           1       0.77      0.73      0.75        74

    accuracy                           0.80       179
   macro avg       0.79      0.79      0.79       179
weighted avg       0.80      0.80      0.80       179



COMPARISON OF LOGISTIC REGRESSION VS RANDOM FOREST CLASSIFIER

In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

rf_model=RandomForestClassifier(n_estimators=100,max_depth=None,random_state=42)
rf_model.fit(X_train,y_train)
y_pred_rf=rf_model.predict(X_test)
accuracy_rf=accuracy_score(y_test,y_pred_rf)
print(f"random forest accurscy : {accuracy_rf:.2f}")
print("confusion matrix:")
print(confusion_matrix(y_test,y_pred_rf))
print('\nClassification report:')
print(classification_report(y_test,y_pred_rf))



random forest accurscy : 0.84
confusion matrix:
[[93 12]
 [16 58]]

Classification report:
              precision    recall  f1-score   support

           0       0.85      0.89      0.87       105
           1       0.83      0.78      0.81        74

    accuracy                           0.84       179
   macro avg       0.84      0.83      0.84       179
weighted avg       0.84      0.84      0.84       179





Random Forest works better because it handles non linearities and feature interactions better then Logestic Regression.

